<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fhybrid-models/models/hybrid_models_ST-HybridWaveStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# -*- coding: utf-8 -*-
"""
Entrenamiento Multi‐rama con GRU encoder–decoder y Transformer para low,
validación y forecast parametrizables, meta‐modelo XGBoost,
paralelización, trazabilidad y límites del departamento de Boyacá.
"""

# 0) Supresión de warnings irrelevantes
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from cartopy.io import DownloadWarning
warnings.filterwarnings("ignore", category=DownloadWarning)

# 1) Parámetros configurables
INPUT_WINDOW   = 60          # número de meses en la ventana de entrada
OUTPUT_HORIZON = 3           # meses de validación y forecast
REF_DATE       = "2025-03"   # fecha de referencia (yyyy-mm)

# 2) Detectar entorno (Local / Colab)
import sys
from pathlib import Path
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    BASE_PATH = Path("/content/drive/MyDrive/ml_precipitation_prediction")
    !pip install -q xarray netCDF4 optuna seaborn cartopy xgboost ace_tools_open
else:
    BASE_PATH = Path.cwd()
    for p in [BASE_PATH, *BASE_PATH.parents]:
        if (p/".git").exists():
            BASE_PATH = p
            break
print(f"▶️ Base path: {BASE_PATH}")

# 3) Rutas y logger
import logging
MODEL_DIR   = BASE_PATH/"models"/"output"/"trained_models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FEATURES_NC = BASE_PATH/"models"/"output"/"features_fusion_branches.nc"
FULL_NC     = BASE_PATH/"data"/"output"/"complete_dataset_with_features_with_clusters_elevation_with_windows.nc"
SHP_USER    = Path("/mnt/data/MGN_Departamento.shp")
BOYACA_SHP  = SHP_USER if SHP_USER.exists() else BASE_PATH/"data"/"input"/"shapes"/"MGN_Departamento.shp"
RESULTS_CSV = MODEL_DIR/f"metrics_w{OUTPUT_HORIZON}_ref{REF_DATE}.csv"
IMAGE_DIR   = MODEL_DIR/"images"
IMAGE_DIR.mkdir(exist_ok=True)
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)

# 4) Imports principales
import numpy            as np
import pandas           as pd
import xarray           as xr
import geopandas        as gpd
import matplotlib.pyplot as plt
import imageio.v2       as imageio
import cartopy.crs      as ccrs
from sklearn.preprocessing import StandardScaler
import psutil
from joblib import cpu_count
import tensorflow       as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add, Dense, Flatten, RepeatVector, Input, TimeDistributed, GRU
from tensorflow.keras.models import Model

# 5) Recursos hardware
CORES     = cpu_count()
AVAIL_RAM = psutil.virtual_memory().available / (1024**3)
gpus = tf.config.list_physical_devices("GPU")
USE_GPU = bool(gpus)
if USE_GPU:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    logger.info(f"🖥 GPU disponible: {gpus[0].name}")
else:
    tf.config.threading.set_inter_op_parallelism_threads(CORES)
    tf.config.threading.set_intra_op_parallelism_threads(CORES)
    logger.info(f"⚙ CPU cores: {CORES}, RAM libre: {AVAIL_RAM:.1f} GB")

# 6) Modelos y utilitarios
def evaluate_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mae  = np.mean(np.abs(y_true - y_pred))
    mape = np.mean(np.abs((y_true - y_pred)/(y_true + 1e-5)))*100
    r2   = 1 - np.sum((y_true-y_pred)**2)/np.sum((y_true-np.mean(y_true))**2)
    return rmse, mae, mape, r2

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, Y, batch_size=32):
        self.X, self.Y = X.astype(np.float32), Y.astype(np.float32)
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.X)/self.batch_size))
    def __getitem__(self, idx):
        sl = slice(idx*self.batch_size, (idx+1)*self.batch_size)
        return self.X[sl], self.Y[sl]

def build_gru_ed(input_shape, horizon, n_cells,
                 latent=128, dropout=0.2):
    inp = Input(shape=input_shape)
    x   = GRU(latent, dropout=dropout)(inp)
    x   = RepeatVector(horizon)(x)
    x   = GRU(latent, dropout=dropout, return_sequences=True)(x)
    out = TimeDistributed(Dense(n_cells))(x)
    m   = Model(inp, out)
    m.compile("adam","mse")
    return m

def build_transformer_ed(input_shape, horizon, n_cells,
                         head_size=64, num_heads=4, ff_dim=256, dropout=0.1):
    inp = Input(shape=input_shape)                            # (window, n_feats)
    attn = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inp, inp)
    x    = Add()([inp, attn])
    x    = LayerNormalization(epsilon=1e-6)(x)
    ff   = Dense(ff_dim, activation="relu")(x)
    ff   = Dense(input_shape[-1])(ff)
    x    = Add()([x, ff])
    x    = LayerNormalization(epsilon=1e-6)(x)
    x    = Flatten()(x)
    x    = Dense(horizon * n_cells)(x)
    out  = layers.Reshape((horizon, n_cells))(x)
    m    = Model(inp, out)
    m.compile("adam","mse")
    return m

def build_gru_ed_low(input_shape, horizon, n_cells,
                     latent=256, dropout=0.1, use_transformer=True):
    if use_transformer:
        return build_transformer_ed(input_shape, horizon, n_cells,
                                    head_size=64, num_heads=4, ff_dim=512, dropout=dropout)
    else:
        return build_gru_ed(input_shape, horizon, n_cells,
                            latent=latent, dropout=dropout)

# 7) Carga de datos y shapefile
logger.info("📂 Cargando datasets…")
ds_full = xr.open_dataset(FULL_NC)
ds_feat = xr.open_dataset(FEATURES_NC)
boyaca_gdf = gpd.read_file(BOYACA_SHP)
if boyaca_gdf.crs is None:
    boyaca_gdf.set_crs(epsg=4326, inplace=True)
else:
    boyaca_gdf = boyaca_gdf.to_crs(epsg=4326)

times = ds_full.time.values.astype("datetime64[M]")
ref   = np.datetime64(REF_DATE,"M")
if ref not in times:
    ref = times[-1]
    logger.warning(f"REF_DATE no hallado; usando último mes: {ref}")
idx_ref = int(np.where(times==ref)[0][0])

lat = ds_full.latitude.values
lon = ds_full.longitude.values
METHODS  = ["CEEMDAN","TVFEMD","FUSION"]
BRANCHES = ["high","medium","low"]

all_metrics  = []
preds_store  = {}
true_store   = {}
histories    = {}   # guardar history por modelo

# 8) Bucle principal
for method in METHODS:
    for branch in BRANCHES:
        name = f"{method}_{branch}"
        if name not in ds_feat.data_vars:
            logger.warning(f"⚠ {name} no existe, saltando.")
            continue
        logger.info(f"▶ Procesando {name}")
        try:
            # extraer y aplanar espacialmente
            Xarr = ds_feat[name].values         # (T,ny,nx)
            yarr = ds_full["total_precipitation"].values
            T, ny, nx = Xarr.shape
            n_cells   = ny*nx

            Xfull = Xarr.reshape(T, n_cells)
            yfull = yarr.reshape(T, n_cells)

            # ventanas deslizantes
            Nw = T - INPUT_WINDOW - OUTPUT_HORIZON + 1
            if Nw<=0:
                logger.warning("❌ Ventanas insuficientes.")
                continue

            # construir Xs, ys
            Xs = np.stack([Xfull[i:i+INPUT_WINDOW] for i in range(Nw)], axis=0)      # (Nw, window, n_cells)
            ys = np.stack([yfull[i+INPUT_WINDOW:i+INPUT_WINDOW+OUTPUT_HORIZON]
                           for i in range(Nw)], axis=0)                             # (Nw, horizon, n_cells)

            # para low-branch: agregar sin/cos estacionales
            if branch=="low":
                months = pd.to_datetime(times).month.values
                sin_feat = np.sin(2*np.pi*months/12)
                cos_feat = np.cos(2*np.pi*months/12)
                Ssin = np.stack([sin_feat[i:i+INPUT_WINDOW] for i in range(Nw)], axis=0)
                Scos = np.stack([cos_feat[i:i+INPUT_WINDOW] for i in range(Nw)], axis=0)
                # tile
                sin_tile = np.repeat(Ssin[:,:,None], n_cells, axis=2)
                cos_tile = np.repeat(Scos[:,:,None], n_cells, axis=2)
                Xs = np.concatenate([Xs, sin_tile, cos_tile], axis=2)  # nuevo n_feats
                n_feats = Xs.shape[2]
            else:
                n_feats = n_cells

            # escalado global
            scX = StandardScaler().fit(Xs.reshape(-1,n_feats))
            scY = StandardScaler().fit(ys.reshape(-1,n_cells))
            Xs_s = scX.transform(Xs.reshape(-1,n_feats)).reshape(Xs.shape)
            ys_s = scY.transform(ys.reshape(-1,n_cells)).reshape(ys.shape)

            # determinar índices para validación centrados en REF_DATE
            k_ref = idx_ref - INPUT_WINDOW + 1
            k_ref = max(0, min(k_ref, Nw-1))
            i0    = k_ref - (OUTPUT_HORIZON-1)
            i0    = max(0, min(i0, Nw-OUTPUT_HORIZON))

            # split train / validación
            X_tr = Xs_s[:i0]
            y_tr = ys_s[:i0]
            X_va = Xs_s[i0:i0+OUTPUT_HORIZON]
            y_va = ys_s[i0:i0+OUTPUT_HORIZON]

            # cargar o entrenar modelo
            if branch=="low":
                model_dir = MODEL_DIR/f"{name}_w{OUTPUT_HORIZON}_ref{ref}"
                saved_model_dir = model_dir/"saved_model"
                if saved_model_dir.exists():
                    model = tf.keras.models.load_model(saved_model_dir)
                    logger.info(f"⏩ Cargado low-branch SavedModel: {saved_model_dir}")
                else:
                    model = build_gru_ed_low((INPUT_WINDOW,n_feats),
                                             OUTPUT_HORIZON, n_cells,
                                             latent=256, dropout=0.1,
                                             use_transformer=True)
                    hist = model.fit(
                        DataGenerator(X_tr,y_tr),
                        validation_data=DataGenerator(X_va,y_va),
                        epochs=100,
                        callbacks=[callbacks.EarlyStopping("val_loss",patience=7,restore_best_weights=True)],
                        verbose=1
                    )
                    # guardar
                    model_dir.mkdir(parents=True, exist_ok=True)
                    model.save(saved_model_dir)
                    histories[name] = hist.history
            else:
                model_path = MODEL_DIR/f"{name}_w{OUTPUT_HORIZON}_ref{ref}.h5"
                if model_path.exists():
                    model = tf.keras.models.load_model(model_path)
                    logger.info(f"⏩ Cargado modelo: {model_path}")
                else:
                    model = build_gru_ed((INPUT_WINDOW,n_feats), OUTPUT_HORIZON, n_cells) \
                            if branch=="high" else build_gru_ed((INPUT_WINDOW,n_feats), OUTPUT_HORIZON, n_cells)
                    hist = model.fit(
                        DataGenerator(X_tr,y_tr),
                        validation_data=DataGenerator(X_va,y_va),
                        epochs=100,
                        callbacks=[callbacks.EarlyStopping("val_loss",patience=5,restore_best_weights=True)],
                        verbose=1
                    )
                    model.save(model_path)
                    histories[name] = hist.history

            # — Validación: H=1…H=OUTPUT_HORIZON
            preds_s = model.predict(X_va, verbose=0)   # (H, H, n_cells) or (N_va, H, n_cells)
            # asumimos X_va.shape[0]==OUTPUT_HORIZON
            preds_s = preds_s.reshape(OUTPUT_HORIZON, OUTPUT_HORIZON, n_cells)
            # invertimos escala
            for h in range(OUTPUT_HORIZON):
                date_val = str(times[i0+h+INPUT_WINDOW-1])
                pm = scY.inverse_transform(preds_s[h,0]).reshape(ny,nx)
                tm = scY.inverse_transform(y_va[h,0]).reshape(ny,nx)
                rmse, mae, mape, r2 = evaluate_metrics(tm.ravel(), pm.ravel())
                all_metrics.append({
                    "model":name, "branch":branch,
                    "horizon":h+1, "type":"validation",
                    "date":date_val,
                    "RMSE":rmse,"MAE":mae,"MAPE":mape,"R2":r2
                })
                preds_store [(name,date_val)] = pm
                true_store  [(name,date_val)] = tm

            # — Forecast tras REF_DATE
            X_fc = Xs_s[k_ref:k_ref+1]
            fc_s = model.predict(X_fc, verbose=0)[0]    # (H, n_cells)
            FC   = scY.inverse_transform(fc_s)
            for h in range(OUTPUT_HORIZON):
                date_fc = str(times[idx_ref] + np.timedelta64(h+1,'M'))
                all_metrics.append({
                    "model":name, "branch":branch,
                    "horizon":h+1, "type":"forecast",
                    "date":date_fc,
                    "RMSE":np.nan,"MAE":np.nan,"MAPE":np.nan,"R2":np.nan
                })
                preds_store[(name,date_fc)] = FC[h].reshape(ny,nx)

        except Exception:
            logger.exception(f"‼ Error en {name}, continúo…")
            continue

# 9) Guardar métricas y mostrar tabla
dfm = pd.DataFrame(all_metrics)
dfm.to_csv(RESULTS_CSV, index=False)
import ace_tools_open as tools
tools.display_dataframe_to_user(name=f"Metrics_w{OUTPUT_HORIZON}_ref{ref}", dataframe=dfm)

# 10) Curvas de entrenamiento
for name,hist in histories.items():
    plt.figure(figsize=(6,4))
    plt.plot(hist["loss"], label="train")
    plt.plot(hist["val_loss"], label="val")
    plt.title(f"Loss curve: {name}")
    plt.xlabel("Epoch"); plt.ylabel("MSE")
    plt.legend(); plt.show()

# 11) Mapas 3×3 validación H=1
dates_va = sorted({d for (_,d) in preds_store if
                   any(r for r in all_metrics if r["date"]==d and r["type"]=="validation")})[:OUTPUT_HORIZON]
for date_val in dates_va:
    # gather global vmin/vmax
    arrs = [preds_store[(f"{m}_{b}",date_val)].ravel()
            for m in METHODS for b in BRANCHES
            if (f"{m}_{b}",date_val) in preds_store]
    vmin = np.min(arrs); vmax = np.max(arrs)
    fig, axs = plt.subplots(3,3, figsize=(12,12),
                             subplot_kw={"projection":ccrs.PlateCarree()})
    fig.suptitle(f"Validación H=1, {date_val}", fontsize=16)
    for i, b in enumerate(BRANCHES):
        for j, m in enumerate(METHODS):
            ax = axs[i,j]
            ax.add_geometries(boyaca_gdf.geometry, ccrs.PlateCarree(),
                              edgecolor="black", facecolor="none", linewidth=1)
            key = (f"{m}_{b}", date_val)
            if key in preds_store:
                pcm = ax.pcolormesh(lon, lat, preds_store[key],
                                    vmin=vmin, vmax=vmax,
                                    transform=ccrs.PlateCarree(), cmap="Blues")
            ax.set_title(f"{m}_{b}")
    cb = fig.colorbar(pcm, ax=axs, orientation="horizontal", fraction=0.05, pad=0.04,
                      label="Precipitación (mm)")
    fig.savefig(IMAGE_DIR/f"val_H1_{date_val}.png", dpi=150)
    plt.show()

    # MAPE
    arrs_mape = []
    for (m,b) in [(m,b) for m in METHODS for b in BRANCHES]:
        key = (f"{m}_{b}",date_val)
        if key in preds_store and key in true_store:
            mape_map = np.clip(np.abs((true_store[key]-preds_store[key])/(true_store[key]+1e-5))*100,0,200)
            arrs_mape.append(mape_map.ravel())
    vmin2, vmax2 = 0, np.max(arrs_mape)
    fig, axs = plt.subplots(3,3, figsize=(12,12),
                             subplot_kw={"projection":ccrs.PlateCarree()})
    fig.suptitle(f"MAPE H=1, {date_val}", fontsize=16)
    for i, b in enumerate(BRANCHES):
        for j, m in enumerate(METHODS):
            ax = axs[i,j]
            ax.add_geometries(boyaca_gdf.geometry, ccrs.PlateCarree(),
                              edgecolor="black", facecolor="none", linewidth=1)
            key = (f"{m}_{b}", date_val)
            if key in preds_store and key in true_store:
                mape_map = np.clip(np.abs((true_store[key]-preds_store[key])/(true_store[key]+1e-5))*100,0,200)
                pcm2 = ax.pcolormesh(lon, lat, mape_map,
                                     vmin=vmin2, vmax=vmax2,
                                     transform=ccrs.PlateCarree(), cmap="Reds")
            ax.set_title(f"{m}_{b}")
    cb2 = fig.colorbar(pcm2, ax=axs, orientation="horizontal", fraction=0.05, pad=0.04,
                       label="MAPE (%)")
    fig.savefig(IMAGE_DIR/f"mape_H1_{date_val}.png", dpi=150)
    plt.show()

# 12) Meta‐modelo XGBoost (low, h=3)
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_meta, y_meta = [], []
for date in dates_va:
    keys = [(f"{m}_low", date) for m in METHODS]
    if all(k in preds_store for k in keys):
        arrs = [preds_store[k].ravel() for k in keys]
        X_meta.append(np.vstack(arrs).T)
        y_meta.append(true_store[keys[0]].ravel())
X_meta = np.concatenate(X_meta, axis=0)
y_meta = np.concatenate(y_meta, axis=0)

Xtr, Xte, ytr, yte = train_test_split(X_meta, y_meta, test_size=0.2, random_state=42)
xgb = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=4, n_jobs=-1)
xgb.fit(Xtr, ytr)
yhat = xgb.predict(Xte)
rmse_meta = np.sqrt(mean_squared_error(yte,yhat))
print(f"Meta‐modelo XGB (low, h=3) RMSE: {rmse_meta:.3f}")

plt.figure(figsize=(5,5))
plt.scatter(yte, yhat, alpha=0.3, s=2)
lims = [min(yte.min(),yhat.min()), max(yte.max(),yhat.max())]
plt.plot(lims, lims, 'k--')
plt.xlabel("True"); plt.ylabel("Predicted")
plt.title("Meta‐modelo XGB (low, h=3)")
plt.show()


2025-05-19 17:37:19,690 INFO ⚙ CPU cores: 10, RAM libre: 3.0 GB
2025-05-19 17:37:19,691 INFO 📂 Cargando datasets…
2025-05-19 17:37:19,725 WARNING REF_DATE no hallado; usando último mes: 2025-02
2025-05-19 17:37:19,726 INFO ▶ Procesando CEEMDAN_high


▶️ Base path: /Users/riperez/Conda/anaconda3/envs/precipitation_prediction/github.com/ml_precipitation_prediction
Epoch 1/100


/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 105ms/step - loss: 1.0134 - val_loss: 0.8062
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.7888 - val_loss: 0.7200
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.6007 - val_loss: 0.7267
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.5893 - val_loss: 0.7551
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.4737 - val_loss: 0.7128
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.4442 - val_loss: 0.6976
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.4166 - val_loss: 0.7586
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.4003 - val_loss: 0.6515
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.3680 - val_loss: 0.8289
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.3805 - val_loss: 0.7004
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.3930 - val_loss: 0.8003
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.

2025-05-19 17:37:41,056 WARNING You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-05-19 17:37:41,372 ERROR ‼ Error en CEEMDAN_high, continúo…
Traceback (most recent call last):
  File "/var/folders/83/c6n8lktn4qx_fwp7ksllkkhn0dhtn2/T/ipykernel_42911/781569034.py", line 272, in <module>
    pm = scY.inverse_transform(preds_s[h,0]).reshape(ny,nx)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/sklearn/preprocessing/_data.py", line 1106, in inverse_transform
    X = check_array(
        ^^^^^^^^^^^^
  File "/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/sklearn/utils/validation.py", line 1093, in check_array
    raise Val

Epoch 1/100


/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - loss: 0.9550 - val_loss: 0.9645
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8952 - val_loss: 1.0057
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.7863 - val_loss: 1.1768
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.6825 - val_loss: 0.8198
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.6333 - val_loss: 1.1406
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.5573 - val_loss: 0.9944
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.5086 - val_loss: 1.0982
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.4573 - val_loss: 1.4394
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.3940 - val_loss: 1.1079


2025-05-19 17:37:54,450 WARNING You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
2025-05-19 17:37:54,776 ERROR ‼ Error en CEEMDAN_medium, continúo…
Traceback (most recent call last):
  File "/var/folders/83/c6n8lktn4qx_fwp7ksllkkhn0dhtn2/T/ipykernel_42911/781569034.py", line 272, in <module>
    pm = scY.inverse_transform(preds_s[h,0]).reshape(ny,nx)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/sklearn/preprocessing/_data.py", line 1106, in inverse_transform
    X = check_array(
        ^^^^^^^^^^^^
  File "/Users/riperez/Conda/anaconda3/envs/precipitation_prediction/lib/python3.12/site-packages/sklearn/utils/validation.py", line 1093, in check_array
    raise V

: 